In [1]:
%load_ext autoreload
%autoreload 2
%cd -q ..

import pickle
import importlib
import esparto
import optuna
import numpy as np
import pandas as pd
from evoaug_tf import evoaug, augment
from src.diff_expression_model import get_model, get_siamese_model, post_hoc_conjoining, get_auroc
from skopt.utils import use_named_args

2025-05-21 16:01:16.218566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-21 16:01:16.342443: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-21 16:01:16.342491: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-05-21 16:01:18.820216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-

## Prepare the data

For now we start from this data pickle as I'm not aware how Helder did DE analysis and generated the labels

In [2]:
ppath = "data/dataset_solid_chrome.csv"
data = pd.read_csv(ppath)

In [3]:
data["set"].value_counts()

train    1900
valid     257
test      241
Name: set, dtype: int64

To show how to subdivide the dataset into train-test split we only take the following columns

In [4]:
dataset = data.reset_index()
dataset = dataset[["geneID", "Category", "GeneFamily", "seqs"]]
dataset.rename(columns={"geneID":"GeneID", "Category":"Label", "seqs": "Sequence"}, inplace=True)

In [5]:
dataset

,GeneID,Label,GeneFamily,Sequence
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...
...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...


### One-hot-encode & reverse-complement

In [6]:
from src.prepare_dataset import one_hot_encode_series, reverse_complement_series, reverse_complement_sequence
dataset["One_hot_encoded"] = one_hot_encode_series(dataset["Sequence"])

In [7]:
dataset["RC_one_hot_encoded"] = reverse_complement_series(dataset["One_hot_encoded"])

In [8]:
dataset

,GeneID,Label,GeneFamily,Sequence,One_hot_encoded,RC_one_hot_encoded
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,...","[[0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,..."
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,...","[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1,..."
...,...,...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,..."
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...,"[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,..."


### Train-test split

In [9]:
from src.prepare_dataset import grouped_shuffle_split
train_df, test_df = grouped_shuffle_split(dataset, dataset["GeneFamily"], 0.2)

In [10]:
print(f"Length of training set: {train_df.shape[0]}")
print(f"Length of test set: {test_df.shape[0]}")

Length of training set: 1900
Length of test set: 498


## Cross validation

We first initialize the object that are static throughout the cross validation

In [11]:
augment_list = [
    augment.RandomRC(rc_prob=0.5),
    augment.RandomInsertionBatch(insert_min=0, insert_max=20),
    augment.RandomDeletion(delete_min=0, delete_max=30),
    augment.RandomTranslocationBatch(shift_min=0, shift_max=20),
    augment.RandomMutation(mutate_frac=0.05),
    augment.RandomNoise()
]

2025-05-21 16:01:53.835082: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-05-21 16:01:53.835114: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-05-21 16:01:53.835135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-vibcompute-1.vib.local): /proc/driver/nvidia/version does not exist
2025-05-21 16:01:53.835592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# early stopping callback
import tensorflow as tf

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
											patience=20,
											verbose=1,
											mode='min',
											restore_best_weights=True)
# reduce learning rate callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
												factor=0.1,
												patience=5,
												min_lr=1e-7,
												mode='min',
												verbose=1)
callbacks = [early_stopping_callback, reduce_lr]

In [13]:
input_shape = train_df["One_hot_encoded"].iloc[0].shape

We prepare the data

In [14]:
def get_input_and_labels(df):
	ohe_np = np.stack(df["One_hot_encoded"])
	rc_np = np.stack(df["RC_one_hot_encoded"])

	x = np.append(ohe_np, rc_np, axis=0)
	x = x.astype('float32')
	y = np.append(df["Label"], df["Label"])
	return x, y

In [15]:
X, Y = get_input_and_labels(train_df)

We will now create the groups. Because we are using both forward and reverse complement we have to concat them with itself

In [16]:
groups = pd.concat([train_df["GeneFamily"], train_df["GeneFamily"]], axis = 0) 

Initialize the splitter

In [17]:
from sklearn.model_selection import GroupKFold

group_kfold = GroupKFold(n_splits=5)

# Define metric containers
loss_list = []
accuracy_list = []
auroc_list = []
auprc_list = []
true_positive_list = []

for i, (train_index, validation_index) in enumerate(group_kfold.split(X, Y, groups)):
	x_train = X[train_index]
	y_train = Y[train_index]

	x_val = X[validation_index]
	y_val = Y[validation_index]

	model = get_model(input_shape=input_shape, perform_evoaug=True, augment_list=augment_list, learning_rate=0.001)

	# We add validation here, because one of the callbacks relies on val_loss metric
	model.fit(x_train,
			y_train,
			epochs=100,	
			batch_size=100,
			validation_data=(x_val, y_val),
			callbacks=callbacks
			)
	scores = model.evaluate(x_val, y_val, verbose=0)
	print(f'Score for fold {i}: Loss of {scores[0]}; accuracy of {scores[1]}; auroc of {scores[2]}; auprc of {scores[3]}; TP of {scores[4]}')
	
	loss_list.append(scores[0])
	accuracy_list.append(scores[1])
	auroc_list.append(scores[2])
	auprc_list.append(scores[3])
	true_positive_list.append(scores[4])
	

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_list)):
  print('------------------------------------------------------------------------')
  print(f'Score for fold {i}: Loss of {loss_list[i]}; accuracy of {accuracy_list[i]}; AUROC of {auroc_list[i]}; AUPRC of {auprc_list[i]}; TP of {true_positive_list[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Loss: {np.mean(loss_list)} (+- {np.std(loss_list)})')
print(f'> Auroc: {np.mean(auroc_list)}')
print('------------------------------------------------------------------------')

Epoch 1/100
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
31/31 [==============================] - 7s 88ms/step - loss: 0.6944 - acc: 0.5089 - auROC: 0.5067 - auPRC: 0.5007 - true_positives: 591.0000 - precision: 0.5021 - recall: 0.3945 - val_loss: 0.6961 - val_acc: 0.4974 - val_auROC: 0.4918 - val_auPRC: 0.4950 - val_true_positives: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 2/100
31/31 [==============================] - 2s 61ms/step - loss: 0.6940 - acc: 0.5076 - auROC: 0.5046 - auPRC: 0.4978 - true_positives: 342.0000 - precision: 0.5007 - recall: 0.2283 - val_loss: 0.6939 - val_acc: 0.4974 - val_auROC: 0.5014 - val_auPRC: 0.4953 - val_true_positives: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 3/100
31/31 [==============================] - 2s 62ms/step

## Test set

We will train a model on all the training data

In [18]:
model = get_model(input_shape=input_shape, perform_evoaug=True, augment_list=augment_list, learning_rate=0.001)
history = model.fit(X,
					Y,
					epochs=100,
					batch_size=100,
					)

Epoch 1/100
38/38 [==============================] - 5s 57ms/step - loss: 0.6948 - acc: 0.4937 - auROC: 0.4928 - auPRC: 0.4885 - true_positives_5: 960.0000 - precision_5: 0.4888 - recall_5: 0.5106
Epoch 2/100
38/38 [==============================] - 2s 58ms/step - loss: 0.6946 - acc: 0.4900 - auROC: 0.4870 - auPRC: 0.4861 - true_positives_5: 830.0000 - precision_5: 0.4831 - recall_5: 0.4415
Epoch 3/100
38/38 [==============================] - 2s 56ms/step - loss: 0.6940 - acc: 0.4911 - auROC: 0.4852 - auPRC: 0.4881 - true_positives_5: 584.0000 - precision_5: 0.4779 - recall_5: 0.3106
Epoch 4/100
38/38 [==============================] - 2s 59ms/step - loss: 0.6935 - acc: 0.5074 - auROC: 0.5035 - auPRC: 0.4993 - true_positives_5: 590.0000 - precision_5: 0.5034 - recall_5: 0.3138
Epoch 5/100
38/38 [==============================] - 2s 56ms/step - loss: 0.6928 - acc: 0.5216 - auROC: 0.5251 - auPRC: 0.5100 - true_positives_5: 699.0000 - precision_5: 0.5232 - recall_5: 0.3718
Epoch 6/100
38/

We perform post-hoc conjoing

In [19]:
siamese_model = get_siamese_model(model.model)

Prepare the test set data

In [20]:
x_test = np.stack(test_df["One_hot_encoded"])
x_test_rc = np.stack(test_df["RC_one_hot_encoded"])

In [21]:
y_test = test_df["Label"].to_numpy()

As we used evo aug in our model all the sequences which were trained on, were of length 620. We use the evoaug padding function to pad the test set to the same length

In [22]:
x_test = model._pad_end(x_test)
x_test_rc = model._pad_end(x_test_rc)

In [23]:
predictions_categories, predictions = post_hoc_conjoining(siamese_model, x_test, x_test_rc)

16/16 [==============================] - 0s 8ms/step


In [24]:
get_auroc(y_test, predictions)

0.7830083520460089